<a href="https://colab.research.google.com/github/TarteTatin1212/face_recognition_script2/blob/main/face_recognition_script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
os.chdir('/content/drive/MyDrive/Horita_Semi/Senior_Project')


In [15]:
!pwd  # 現在のカレントディレクトリを表示
!ls   # カレントディレクトリのファイルリストを表示


/content/drive/MyDrive/Horita_Semi/Senior_Project
api_key.txt			merged_data.csv		       Untitled0.ipynb
contents_search.ipynb		merge_face_and_metadata.ipynb  youtube_thumbnail_downloader.ipynb
data_analytics.ipynb		models			       youtube_utils.ipynb
datasets			__pycache__		       YuNetFaceDetectionResults
FaceRecognitionResults		saved_data		       yunet.ipynb
face_recognition_script2.ipynb	senior_project_main.ipynb      yunet.py
face_recognition_script.ipynb	thumbnails		       課題点.gdoc


In [16]:
import nbimporter
import glob
import numpy as np
import cv2 as cv
from yunet import YuNet  # yunet.pyからYuNetクラスをインポート

In [17]:
# バウンディングボックスと信頼度スコアを描画する関数
def visualize(image, results, box_color=(255, 0, 0), text_color=(255, 0, 0)):
    # 引数で渡された画像のコピーを作成
    output = image.copy()

    # 検出結果(results)の各要素に対してループ処理
    for det in results:
        # detからバウンディングボックスの座標を抽出し、整数型にキャスト
        bbox = det[0:4].astype(np.int32)

        # 抽出したバウンディングボックスの座標を使用して、画像に矩形を描画
        cv.rectangle(output,
                     (bbox[0], bbox[1]),  # 矩形の左上の座標
                     (bbox[0] + bbox[2], bbox[1] + bbox[3]),  # 矩形の右下の座標
                     box_color,  # 矩形の色
                     2)  # 矩形の線の太さ

        # detの最後の要素から信頼度スコアを抽出
        conf = det[-1]

        # 抽出した信頼度スコアを画像にテキストとして描画
        cv.putText(output,
                   '{:.4f}'.format(conf),  # テキストとして描画する信頼度スコア
                   (bbox[0], bbox[1] + 12),  # テキストの位置（バウンディングボックスの左上から少し下）
                   cv.FONT_HERSHEY_DUPLEX,  # フォントスタイル
                   0.5,  # フォントサイズ
                   text_color)  # テキストの色

    # 処理が完了した画像を返す
    return output


In [18]:
# YuNetモデルのファイルパスを指定
# このパスはGoogle Drive内の特定のディレクトリにあるYuNetモデルへのパス
model_path = '/content/drive/MyDrive/Horita_Semi/Senior_Project/models/face_detection_yunet_2023mar.onnx'

# YuNetモデルのインスタンスを作成
model = YuNet(
    modelPath=model_path,  # モデルファイルへのパス
    inputSize=[320, 320],  # モデルの入力サイズ。ここでは320x320ピクセル
    confThreshold=0.9,     # 検出信頼度の閾値。0.9は高い信頼度を意味する
    nmsThreshold=0.3,      # 非最大抑制（Non-Maximum Suppression）の閾値。0.3は標準的な値
    topK=5000              # 検出された顔の最大数。ここでは5000個まで検出可能
)


In [21]:
# 画像が保存されているディレクトリのパスを指定
directory_path = '/content/drive/MyDrive/Horita_Semi/Senior_Project/FaceRecognitionResults/UnrecognizedFaces'

# 指定されたディレクトリ内のすべてのJPEG画像ファイルのリストを取得
image_files = glob.glob(
    os.path.join(directory_path, '*.jpg')
)

# 取得した画像ファイルのリストに対してループ処理
for file in image_files:
    # 画像ファイルを読み込み
    image = cv.imread(file)

    # 画像が正しく読み込まれなかった場合の処理
    if image is None:
        print(f"Failed to load image {file}")
        continue

    # 画像処理を試みる
    try:
        # 画像の高さと幅を取得
        h, w, _ = image.shape

        # YuNetモデルに画像サイズを設定
        model.setInputSize([w, h])

        # モデルを使って画像内の顔を検出
        results = model.infer(image)

        # 検出された顔の数を出力
        # print('{} faces detected in {}.'.format(len(results), file))

        # 検出された顔がある場合の処理
        if len(results) > 0:
            # 検出された顔にバウンディングボックスを描画
            image_with_boxes = visualize(image, results)

            # 加工された画像を保存するパスを設定
            save_path = os.path.join(
                '/content/drive/MyDrive/Horita_Semi/Senior_Project/FaceRecognitionResults/RecognizedFace',
                os.path.basename(file)
            )

            # 加工された画像を指定されたパスに保存
            cv.imwrite(save_path, image_with_boxes)

    # 画像処理中にエラーが発生した場合の処理
    except Exception as e:
        print(f"Error processing image {file}: {e}")
